In [1]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
class Textos:
    
    def __init__(self, clases, webs):
        self.clases = clases
        self.lista_dataset = []
        self.crear_dataframe(clases, webs)

    def crear_dataframe(self, clases, webs):
        self.lista_dataset.append('TextoCrudo')
        self.lista_dataset.append('TextoLimpio')

        for i in range(1, clases + 1):
            self.lista_dataset.append('Clase' + str(i))

        self.lista_dataset.append('Predicción')
        self.lista_dataset.append('CLASE')
        self.dataframe = pd.DataFrame(columns=self.lista_dataset)
        self.webs = pd.ExcelFile(webs)
        df = self.webs.parse(self.webs.sheet_names[0])
        self.__clases_df = set(df["CÓDIGO"].value_counts().index)

        for i in df.index:
            print(f"Creando Dataframe {i+1} de {len(df.index)}")
            url = df["URL"][i]
            req = get(url)
            statusCode = req.status_code
            html = BeautifulSoup(req.text, "html.parser")

            if statusCode == 200:
                paragraphs = html.select("p")
                cont = 0

                for para in paragraphs:
                    if len(para.text) < 15:
                        continue
                    cont += 1
                    self.dataframe = self.dataframe.append({'TextoCrudo': para.text, 'CLASE': df["CÓDIGO"][i]}, ignore_index=True)

                clear_output(wait=True)

    def limpiar(textoCrudo):
        for i in range(100):
            textoCrudo = textoCrudo.replace("[" + str(i) + "]", "")
        
        for i in ["a","b","c"]:
            textoCrudo = textoCrudo.replace("[" + i + "]", "")
        
        stopwords_english = stopwords.words('english')
        stop        = stopwords.words('english')
        tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
        stemmer = PorterStemmer()    
        texto_tokens = tokenizer.tokenize(textoCrudo)
        sufijos_fecha = ["bc","ac"]

        for i in range(len(texto_tokens.copy())):
            if i > len(texto_tokens) - 1:
                break
            if i !=0:
                if texto_tokens[i] in sufijos_fecha and texto_tokens[i - 1].isnumeric():
                    texto_tokens[i] = texto_tokens[i - 1] + texto_tokens[i]     
                    texto_tokens.pop(i - 1)
                           
        texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation]  
        texto_stem   = [stemmer.stem(word) for word in texto_clean] 
        return texto_stem       
      

In [32]:
clases = 5
webs = "webs.xlsx"
textos = Textos(clases, webs)

Creando Dataframe 89 de 89


In [33]:
textos.dataframe

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,"During his youth, Alexander was tutored by Ari...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Alexander's legacy includes the cultural diffu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,"Alexander was born in Pella, the capital of th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
6059,There are nevertheless a number of contemporar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
6060,Evidence suggests big-game hunter-gatherers cr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
6061,Hunter-gatherers would eventually flourish all...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
6062,The Archaic period in the Americas saw a chang...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [36]:
textos.dataframe.to_csv("Historia.csv")

In [50]:
dataframe = pd.read_csv("Historia.csv")
dataframe = dataframe.iloc[:,1:]
dataframe

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,"During his youth, Alexander was tutored by Ari...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"Alexander endeavoured to reach the ""ends of th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Alexander's legacy includes the cultural diffu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,"Alexander was born in Pella, the capital of th...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
6059,There are nevertheless a number of contemporar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
6060,Evidence suggests big-game hunter-gatherers cr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
6061,Hunter-gatherers would eventually flourish all...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
6062,The Archaic period in the Americas saw a chang...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [51]:
dataframe.iloc[4029,0]

"In May 1798, Bonaparte was elected a member of the French Academy of Sciences. His Egyptian expedition included a group of 167 scientists, with mathematicians, naturalists, chemists, and geodesists among them. Their discoveries included the Rosetta Stone, and their work was published in the Description de l'Égypte in 1809.[68]\n"

In [52]:
dataframe.describe()

,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6064.000000
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.978232
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.418588
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000


In [53]:
crudo = "10 bc bcc Hola 10 bcc chau 5 bc 555 bc bcc 10 7000 ac"
limpio = Textos.limpiar(crudo)
print(type(limpio))
limpio

<class 'list'>


['10bc',
 'bcc',
 'hola',
 '10',
 'bcc',
 'chau',
 '5bc',
 '555bc',
 'bcc',
 '10',
 '7000ac']

In [55]:
dataframe["LTLimpio"], dataframe["LTCrudo"] = 0, 0

for i in dataframe.index:
    dataframe["TextoLimpio"][i] = Textos.limpiar(dataframe["TextoCrudo"][i])
    dataframe["LTCrudo"][i] = len(dataframe["TextoCrudo"][i])
    dataframe["LTLimpio"][i] = len(dataframe["TextoLimpio"][i])

dataframe  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTLimpio,LTCrudo
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",NaN,NaN,NaN,NaN,NaN,NaN,1,78,757
1,"During his youth, Alexander was tutored by Ari...","[youth, alexand, tutor, aristotl, age, 16, phi...",NaN,NaN,NaN,NaN,NaN,NaN,1,74,821
2,"Alexander endeavoured to reach the ""ends of th...","[alexand, endeavour, reach, end, world, great,...",NaN,NaN,NaN,NaN,NaN,NaN,1,58,624
3,Alexander's legacy includes the cultural diffu...,"[alexander', legaci, includ, cultur, diffus, s...",NaN,NaN,NaN,NaN,NaN,NaN,1,87,1007
4,"Alexander was born in Pella, the capital of th...","[alexand, born, pella, capit, kingdom, macedon...",NaN,NaN,NaN,NaN,NaN,NaN,1,46,488
...,...,...,...,...,...,...,...,...,...,...,...
6059,There are nevertheless a number of contemporar...,"[nevertheless, number, contemporari, hunter-ga...",NaN,NaN,NaN,NaN,NaN,NaN,5,86,1011
6060,Evidence suggests big-game hunter-gatherers cr...,"[evid, suggest, big-gam, hunter-gather, cross,...",NaN,NaN,NaN,NaN,NaN,NaN,5,54,549
6061,Hunter-gatherers would eventually flourish all...,"[hunter-gather, would, eventu, flourish, ameri...",NaN,NaN,NaN,NaN,NaN,NaN,5,71,724
6062,The Archaic period in the Americas saw a chang...,"[archaic, period, america, saw, chang, environ...",NaN,NaN,NaN,NaN,NaN,NaN,5,64,695


In [58]:
dataframe["TextoLimpio"][6063]

['media', 'relat', 'hunter-gather', 'wikimedia', 'common']

In [ ]:
conta = 0
for i in dataframe.index:
    if len(dataframe["TextoLimpio"][i]) < 10:
        conta += 1 
        print(dataframe["TextoLimpio"][i]) 
print(conta)        

['media', 'egypt', '333bc']
['historian', 'egyptologist', 'joann', 'fletcher', 'said', 'alexand', 'blond', 'hair', '190']
['greek', 'antholog', 'poem', 'refer', 'alexand', '280', '281']
['theori', 'realiti', 'led', 'radic', 'differ', 'approach', 'scienc']
['roman', 'creat', 'fireproof', 'wood', 'coat', 'wood', 'alum']
['roman', 'cavalri', 'saddl', 'four', 'horn', 'believ', 'copi', 'celtic', 'peopl']
['immedi', 'caus', 'rebellion', 'gross', 'mistreat', 'roman', 'tacitu', 'wrote']
['cassiu', 'dio', 'wrote']
['region', 'caesar', 'never', 'knew', 'thi', 'poster', 'shall', 'sway']
['suspect', 'children']
['grandchild', 'julia', 'pompey', 'dead', 'sever', 'day', 'unnam', '145']
['work', 'histor', 'attribut', 'caesar', 'authorship', 'doubt']
['bust', 'napl', 'nation', 'archaeolog', 'museum', 'photograph', 'publish', '1902']
['bust', 'nation', 'archaeolog', 'museum', 'napl']
['bust', 'juliu', 'caesar', 'british', 'museum']
['modern', 'bronz', 'statu', 'juliu', 'caesar', 'rimini', 'itali']
['at

In [ ]:
dataframe = dataframe[dataframe["LTLimpio"] > 7]
dataframe

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE,LTCrudo,LTLimpio
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,"[alexand, iii, macedon, greek, αλέξανδρος, γʹ,...",NaN,NaN,NaN,NaN,NaN,NaN,1,757,78
1,"During his youth, Alexander was tutored by Ari...","[youth, alexand, tutor, aristotl, age, 16, phi...",NaN,NaN,NaN,NaN,NaN,NaN,1,821,74
2,"Alexander endeavoured to reach the ""ends of th...","[alexand, endeavour, reach, end, world, great,...",NaN,NaN,NaN,NaN,NaN,NaN,1,624,58
3,Alexander's legacy includes the cultural diffu...,"[alexander', legaci, includ, cultur, diffus, s...",NaN,NaN,NaN,NaN,NaN,NaN,1,1007,87
4,"Alexander was born in Pella, the capital of th...","[alexand, born, pella, capit, kingdom, macedon...",NaN,NaN,NaN,NaN,NaN,NaN,1,488,46
...,...,...,...,...,...,...,...,...,...,...,...
6058,Some agriculturalists also regularly hunt and ...,"[agriculturalist, also, regularli, hunt, gathe...",NaN,NaN,NaN,NaN,NaN,NaN,5,592,54
6059,There are nevertheless a number of contemporar...,"[nevertheless, number, contemporari, hunter-ga...",NaN,NaN,NaN,NaN,NaN,NaN,5,1011,86
6060,Evidence suggests big-game hunter-gatherers cr...,"[evid, suggest, big-gam, hunter-gather, cross,...",NaN,NaN,NaN,NaN,NaN,NaN,5,549,54
6061,Hunter-gatherers would eventually flourish all...,"[hunter-gather, would, eventu, flourish, ameri...",NaN,NaN,NaN,NaN,NaN,NaN,5,724,71


In [ ]:
dataframe.to_csv("Historia.csv")